In [1]:
# Imports
import pdfplumber
import time
import csv
from fuzzywuzzy import fuzz
from bot import get_assistant_response, instructions
from app import assistant, client
from api import *
from sentence_transformers import SentenceTransformer, util
import time
import os
from eval_utils import *

c:\Users\abdar\anaconda3\envs\Case4BCwDS\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
2025-05-22 21:18:33.570 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-22 21:18:36.141 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-22 21:18:36.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-22 21:18:36.143 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script wi

8J6pTdfaGgA5r193UVLsBshUspqwNpal42Jse1aHaok1cWNTLpRkJQQJ99BDACYeBjFXJ
'https://ai-bcds.openai.azure.com/


c:\Users\abdar\anaconda3\envs\Case4BCwDS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
qa_pairs = extract_qa_pairs_from_pdf('./docs/Documents for training and evaluation-20250507\Questions_Answers_Censored.pdf')

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


# String Similarity

In [3]:
# --- Run Evaluation ---
evaluate_string_and_save(
    "./docs/Documents for training and evaluation-20250507/Questions_Answers_Censored.pdf",
    qa_pairs
)



--- Q1 ---
Question: O Fidelidade Savings é mais indicado para que perfil de risco?

Answer: O **Fidelidade My Savings** é um produto que se destina a investidores com um perfil de risco **baixo a moderado**. Este produto é ideal para aqueles que procuram uma forma segura de poupança, com a possibilidade de rentabilidade, mas que não desejam assumir riscos elevados.

Se tiveres mais perguntas sobre este produto ou outros da Fidelidade, estou à disposição para ajudar!

Expected: o Fidelidade Savings é adequado para clientes conservadores, através da sua opção Seguro, com capital e rendimento garantido. Para clientes que gostam moderadamente do risco, através da opção Proteção com garantia de 90% do capital investido no vencimento e exposição ao Índice Multiativos. Para clientes propensos ao risco através da opção Dinâmico com 100% de exposição ao Índice Multiativos. Também está disponível a opção ESG, que investe no índice Multiativos ESG, com risco mas ESG, para clientes com preocupaç

| **Metric**            | **What It Measures**                                                                 |
|-----------------------|--------------------------------------------------------------------------------------|
| **Accuracy**          | Percentage of chatbot answers that closely match the expected answer.               |
| **Similarity Score**  | Fuzzy match score (0–100) comparing chatbot’s answer to the expected answer.        |
| **Match (Yes/No)**    | Whether the answer is considered correct (based on a similarity threshold, e.g. 70).|
| **Response Time**     | Time (in seconds) taken by the chatbot to respond to a question.                    |
| **Average Response**  | Average of all response times — reflects chatbot’s overall speed/efficiency.       |


#### Why String Similarity Evaluation Can Be Problematic

Evaluating chatbot or assistant answers using **string similarity** methods (like fuzzy matching or Levenshtein distance) can be unreliable for several reasons:

- **Surface-Level Matching:** String similarity compares the literal characters or tokens in the text. It does not understand the *meaning* behind the answer. Two answers can be semantically equivalent but phrased very differently, leading to low similarity scores.

- **Sensitive to Minor Differences:** Even small changes like punctuation, synonyms, or word order can drastically reduce the similarity score, unfairly marking correct answers as wrong.

- **Ignores Context and Paraphrasing:** Good language models often rephrase answers or use synonyms, which string similarity methods might penalize despite the answer being correct.

Because of these limitations, string similarity can produce misleading evaluation results, especially in more complex or natural language scenarios.

---

#### Next Step: Evaluating with Semantic Similarity

To better capture the *meaning* behind responses, we will move to **semantic similarity** evaluation. This approach uses embeddings from models like Sentence Transformers to compare the *contextual meaning* of two sentences, rather than just their surface forms.

Semantic similarity can:

- Recognize paraphrases and synonyms
- Be more robust to minor wording differences
- Provide a more accurate measure of whether the chatbot's answer truly matches the expected answer

Next, we will implement and test semantic similarity evaluation using pre-trained sentence embedding models.


# Semantic Similarity

In [4]:
# Load the model ONCE here (in your notebook/session)
model = SentenceTransformer('all-mpnet-base-v2')

# Call your evaluation function, passing the loaded model and the save function
evaluate_semantic_and_save(
    pdf_path='./path/to/your.pdf',
    qa_pairs = qa_pairs,
    model=model,
    threshold=0.7,
    save_results_seq=save_results_seq
)



--- Q1 ---
Question: O Fidelidade Savings é mais indicado para que perfil de risco?
Answer: O produto **Fidelidade My Savings** é mais indicado para investidores com um perfil de risco **conservador**. Este perfil é caracterizado por uma aversão a perdas e uma preferência por investimentos que ofereçam maior segurança e estabilidade, mesmo que isso signifique um potencial de retorno mais baixo.

Se tiveres mais perguntas sobre o produto ou sobre como ele se compara a outros no mercado, estou à disposição para ajudar!
Expected: o Fidelidade Savings é adequado para clientes conservadores, através da sua opção Seguro, com capital e rendimento garantido. Para clientes que gostam moderadamente do risco, através da opção Proteção com garantia de 90% do capital investido no vencimento e exposição ao Índice Multiativos. Para clientes propensos ao risco através da opção Dinâmico com 100% de exposição ao Índice Multiativos. Também está disponível a opção ESG, que investe no índice Multiativos E

# WARNING: for certaint about the results, it should be run at least 15 times

## Summary of the Evaluation Script

This script evaluates the quality of answers generated by an assistant to a set of questions extracted from a PDF file. Instead of relying on simple string matching, it uses **semantic similarity** to compare the assistant's answers to the expected answers.

### Key Steps:
- Extracts question-answer pairs from a PDF.
- Sends each question to the assistant and retrieves its answer.
- Converts both the assistant's answer and the expected answer into semantic embeddings using a pretrained Sentence Transformer model.
- Computes the cosine similarity between the embeddings to measure how close the meanings are.
- Determines if the assistant's answer matches the expected answer based on a similarity threshold.
- Records results, including similarity scores, matches, and response times.
- Saves detailed evaluation results to a CSV file and prints a summary report.

### Benefits:
- Captures meaning rather than exact wording.
- Handles paraphrased or reworded answers effectively.
- Provides a more robust and meaningful assessment of the assistant’s performance.
